In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
Season = cibblbibbl.season.Season
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.register_tournaments()

In [4]:
G.register_matchups()

In [5]:
G.register_achievements()

In [15]:
def tournament_teams(self):
  return {
      Te
      for Mu in self.matchups
      for Te in Mu.teams
      if Mu.excluded == "no"
  } - self.excluded_teams

In [16]:
def season_teams(self):
  return {
      Te for T in self.tournaments.values()
      for Te in tournament_teams(T)
  }

In [12]:
def consecutive_teams(self, past_seasons):
  fromS = self.group.config.get("partnership_introduced")
  if not fromS:
    return set()
  if [self.year_nr, self.nr] < fromS:
    return set()
  S = self
  seasons = set()
  for _ in range(past_seasons):
    if not S:
      return set()
    seasons.add(S)
    S = S.prev
  teams = set(self.group.teams)
  for S in seasons:
      if S.nr == 1:  # skip winter TODO: more sophisticated
          continue
      teams &= season_teams(S)
  return teams

In [17]:
S = max(G.seasons).prev

In [24]:
consecutive_teams(S, 8)

{Team(854257),
 Team(854507),
 Team(854600),
 Team(854678),
 Team(855120),
 Team(855753),
 Team(859097),
 Team(867862),
 Team(872475),
 Team(873349),
 Team(874302),
 Team(874317),
 Team(874452),
 Team(874545),
 Team(874966),
 Team(876774),
 Team(878945),
 Team(879131),
 Team(880378),
 Team(882382),
 Team(885728),
 Team(886002),
 Team(886114),
 Team(888994),
 Team(889232),
 Team(891237),
 Team(891260),
 Team(892248),
 Team(895929),
 Team(896134),
 Team(896228),
 Team(896564),
 Team(896972),
 Team(897353),
 Team(897376),
 Team(897379),
 Team(898574),
 Team(899682),
 Team(900747),
 Team(900800),
 Team(903654)}

In [36]:
s = S
print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in season_teams(s)))}')
print()
while s.prev:
    s = s.prev
    print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in season_teams(s)))}')
    print()

Y5 Summer 854257,854507,854600,854678,855120,855753,859097,864291,867030,867862,871312,871787,872044,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,880378,882382,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379,898507,898574,899682,900747,900800,903654,904377,904411,905886,905924,906026,906259,907600,908411,908529,909759,909848,909966,910344,913702,914538,914744,916031,916217,919573,919662,919838,922608,922672,922869,923094,923140,924366,924370,924405,924441,924841,925504,925668,925779,926021,926025,926053,926362,926596,926991,927862,928360,928392,928542,930089,930778,930817,931222,931786,931901,931935

Y5 Spring 854257,854507,854600,854678,854690,855120,855753,859097,864291,867030,867862,871312,871787,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,880378,882382,883051,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379

In [37]:
s = S
print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in s.teams(True)))}')
print()
while s.prev:
    s = s.prev
    print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in s.teams(True)))}')
    print()

Y5 Summer 854257,854507,854600,854678,855120,855753,859097,864291,867030,867862,871312,871787,872044,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,882382,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379,898507,898574,899682,900747,900800,903654,904377,904411,905886,905924,906026,906259,907600,908411,908529,909759,909848,909966,910344,913702,914538,914744,916031,916217,919573,919662,919838,922608,922672,922869,923094,923140,924366,924370,924405,924441,924841,925504,925668,925779,926021,926025,926053,926362,926596,926991,927862,928360,928392,928542,930089,930778,930817,931222,931786,931901,931935

Y5 Spring 854257,854507,854600,854678,854690,855120,855753,859097,864291,867030,867862,871312,871787,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,880378,882382,883051,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379,898337

In [38]:
S.continuous_activity()

{Team(930817): 1,
 Team(923140): 4,
 Team(910344): 7,
 Team(898574): 9,
 Team(906259): 7,
 Team(919573): 4,
 Team(867862): 16,
 Team(909848): 7,
 Team(879131): 13,
 Team(872475): 16,
 Team(864291): 15,
 Team(922672): 4,
 Team(874545): 14,
 Team(896564): 9,
 Team(931901): 1,
 Team(916031): 5,
 Team(854600): 19,
 Team(855120): 19,
 Team(925779): 3,
 Team(931935): 1,
 Team(899682): 9,
 Team(928360): 2,
 Team(914538): 5,
 Team(872044): 13,
 Team(919662): 4,
 Team(927862): 2,
 Team(908411): 7,
 Team(896134): 10,
 Team(928392): 2,
 Team(900747): 8,
 Team(909966): 7,
 Team(854678): 19,
 Team(926362): 3,
 Team(905886): 7,
 Team(888994): 12,
 Team(924841): 3,
 Team(904377): 8,
 Team(900800): 8,
 Team(905924): 8,
 Team(855753): 19,
 Team(882382): 12,
 Team(924366): 3,
 Team(924370): 3,
 Team(867030): 14,
 Team(904411): 8,
 Team(896228): 10,
 Team(876774): 13,
 Team(908529): 7,
 Team(854257): 19,
 Team(886002): 12,
 Team(924405): 3,
 Team(922869): 4,
 Team(916217): 5,
 Team(880378): 0,
 Team(9269

In [59]:
def continuous_activity(self):
  broken = set()
  d = {Te: 0 for Te in self.teams()}
  this_teams = set(d)
  s = self
  print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in s.teams(True)))}')
  print()
  for Te in s.teams(True):
    d[Te] += 1
  while s.prev:
    s = s.prev
    teams = s.teams(True)
    print(f'Y{s.year.nr} {s.name} {",".join(sorted(str(Te.Id) for Te in s.teams(True)))}')
    for Te in this_teams:
      if Te in broken:
        continue
      if Te in teams:
        d[Te] += 1
      else:
        broken.add(Te)
    print()
  return d

In [60]:
continuous_activity(S)

Y5 Summer 854257,854507,854600,854678,855120,855753,859097,864291,867030,867862,871312,871787,872044,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,882382,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379,898507,898574,899682,900747,900800,903654,904377,904411,905886,905924,906026,906259,907600,908411,908529,909759,909848,909966,910344,913702,914538,914744,916031,916217,919573,919662,919838,922608,922672,922869,923094,923140,924366,924370,924405,924441,924841,925504,925668,925779,926021,926025,926053,926362,926596,926991,927862,928360,928392,928542,930089,930778,930817,931222,931786,931901,931935

Y5 Spring 854257,854507,854600,854678,854690,855120,855753,859097,864291,867030,867862,871312,871787,872475,873349,874302,874317,874452,874545,874966,876774,878945,879131,880378,882382,883051,885728,886002,886114,888994,889232,891237,891260,892248,895929,896134,896228,896564,896972,897353,897376,897379,898337

{Team(930817): 1,
 Team(923140): 4,
 Team(910344): 7,
 Team(898574): 9,
 Team(906259): 7,
 Team(919573): 4,
 Team(867862): 16,
 Team(909848): 7,
 Team(879131): 13,
 Team(872475): 16,
 Team(864291): 5,
 Team(922672): 4,
 Team(874545): 14,
 Team(896564): 9,
 Team(931901): 1,
 Team(916031): 5,
 Team(854600): 19,
 Team(855120): 19,
 Team(925779): 3,
 Team(931935): 1,
 Team(899682): 9,
 Team(928360): 2,
 Team(914538): 5,
 Team(872044): 1,
 Team(919662): 4,
 Team(927862): 2,
 Team(908411): 7,
 Team(896134): 10,
 Team(928392): 2,
 Team(900747): 8,
 Team(909966): 7,
 Team(854678): 19,
 Team(926362): 3,
 Team(905886): 3,
 Team(888994): 12,
 Team(924841): 3,
 Team(904377): 7,
 Team(900800): 8,
 Team(905924): 7,
 Team(855753): 19,
 Team(882382): 12,
 Team(924366): 3,
 Team(924370): 3,
 Team(867030): 5,
 Team(904411): 7,
 Team(896228): 10,
 Team(876774): 13,
 Team(908529): 7,
 Team(854257): 19,
 Team(886002): 12,
 Team(924405): 3,
 Team(922869): 4,
 Team(916217): 5,
 Team(880378): 12,
 Team(926991